In [1]:
DATA_PATH = "../data/processed/02_cleaned_data.pkl"

MLFLOW_TRACKING_URI = "../models/mlruns"
MLFLOW_EXPERIMENT_NAME = "car_price_prediction"

LOG_PATH = "../models/temp/"
LOG_DATA_PKL = "data.pkl"
LOG_MODEL_PKL = "model.pkl"
LOG_METRICS_PKL = "metrics.pkl"

In [2]:
# Load packages
import pandas as pd

from pathlib import Path
import os
import pickle

from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

import mlflow
from mlflow.tracking import MlflowClient

pd.options.display.max_columns = 100

___
## Functions

In [3]:
def calculate_quality(ground_truth, predictions, metric_function, model_name):
    """
    Calculate the quality of the model according to different metric scores
    Input:
        ground_truth: from real observed data
        predictions: the predicted values from the model
        metric_function: the metric score funcrion used to measure performance
    Output:
        A dict of all scores for the given inputs
    """
    quality_score = {}
    quality_score[model_name] = round(metric_function(ground_truth, predictions), 3)
    quality_score = pd.Series(quality_score.values(), index=quality_score.keys())
    
    return quality_score

___
## Itialize
### Create directories:

In [4]:
Path(MLFLOW_TRACKING_URI).mkdir(parents=True, exist_ok=True)
Path(LOG_PATH).mkdir(parents=True, exist_ok=True)

___
## Read data:

In [5]:
df = pd.read_pickle(DATA_PATH)
df.sample(5)

,year,engine_hp,engine_cylinders,transmission_type,msrp,speed,acura,alfa_romeo,aston_martin,audi,bentley,bmw,bugatti,buick,cadillac,chevrolet,chrysler,dodge,ferrari,fiat,ford,genesis,gmc,honda,hummer,hyundai,infiniti,kia,lamborghini,land_rover,lexus,lincoln,lotus,maserati,maybach,mazda,mclaren,mercedes-benz,mitsubishi,nissan,oldsmobile,plymouth,pontiac,porsche,rolls-royce,saab,scion,spyker,subaru,suzuki,tesla,toyota,volkswagen,volvo
5419,27,292.0,4.0,1,10.542601,26.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
10311,26,179.0,4.0,1,9.863238,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6879,27,303.0,6.0,2,10.692990,21.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7830,26,325.0,6.0,1,10.885585,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11501,26,240.0,4.0,2,10.707751,24.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


___
## Splitting data into train and test:

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('msrp', axis=1),
                                                    df[['msrp']],
                                                    test_size=0.2,
                                                    random_state=42)

In [7]:
# quick check of splitted data:
print("X_train shape", X_train.shape)
print("X_test shape", X_test.shape)
X_train.sample(5)

X_train shape (6873, 53)
X_test shape (1719, 53)


,year,engine_hp,engine_cylinders,transmission_type,speed,acura,alfa_romeo,aston_martin,audi,bentley,bmw,bugatti,buick,cadillac,chevrolet,chrysler,dodge,ferrari,fiat,ford,genesis,gmc,honda,hummer,hyundai,infiniti,kia,lamborghini,land_rover,lexus,lincoln,lotus,maserati,maybach,mazda,mclaren,mercedes-benz,mitsubishi,nissan,oldsmobile,plymouth,pontiac,porsche,rolls-royce,saab,scion,spyker,subaru,suzuki,tesla,toyota,volkswagen,volvo
8479,18,127.0,4.0,2,23.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
54,28,248.0,4.0,2,28.5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11285,25,268.0,6.0,2,22.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7298,26,248.0,6.0,2,23.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1152,27,281.0,6.0,2,18.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


___ 
## Trying the base model:

### Intialize MLflow

In [8]:
# Intialize client and experiment
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()

In [9]:
# Check if experiment already exists; if not create one
try:
    mlflow.create_experiment(MLFLOW_EXPERIMENT_NAME)
except:
    print(f'Experiment "{MLFLOW_EXPERIMENT_NAME}" exists at "{mlflow.get_tracking_uri()}"')

In [10]:
exp = client.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
print(exp)

<Experiment: artifact_location='../models/mlruns/0', experiment_id='0', lifecycle_stage='active', name='car_price_prediction', tags={}>


**Now constructing the baseline model**
### Linear Regression:

In [11]:
lr = make_pipeline(MinMaxScaler(),
                   LinearRegression())

lr.fit(X_train.values, y_train.values)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('linearregression', LinearRegression())])

**Evaluate on training data**

In [12]:
# Evaluate on training set:
predictions = pd.DataFrame(lr.predict(X_train.values),
                          columns=y_train.columns)

train_scores = {score.__name__:calculate_quality(y_train, predictions, score, "LinearRegression")
                for score in [r2_score, mean_squared_error, mean_absolute_error]}

train_scores = pd.concat(train_scores, axis=1)
train_scores

,r2_score,mean_squared_error,mean_absolute_error
LinearRegression,0.828,0.023,0.12


In [13]:
mean_train_score = train_scores.mean()
mean_train_score

r2_score               0.828
mean_squared_error     0.023
mean_absolute_error    0.120
dtype: float64

**Evaluate on testing data**

In [14]:
# Evaluate on testing set:
predictions = pd.DataFrame(lr.predict(X_test.values),
                           columns=y_train.columns)

test_scores = {score.__name__:calculate_quality(y_test, predictions, score, "LinearRegression")
                for score in [r2_score, mean_squared_error, mean_absolute_error]}

test_scores = pd.concat(test_scores, axis=1)
test_scores

,r2_score,mean_squared_error,mean_absolute_error
LinearRegression,0.827,0.023,0.121


In [15]:
mean_test_score = test_scores.mean()
mean_test_score

r2_score               0.827
mean_squared_error     0.023
mean_absolute_error    0.121
dtype: float64

___
## Log Run
**Now we save logs in mlflow directory**
### 1. Prepare

In [16]:
# Data details
data_details = {"data_path": DATA_PATH,
                "training_indices": X_train.index.tolist(),
                "test_indices": X_test.index.tolist(),
                "feature_names": X_train.columns.tolist(),
                "target_names": y_train.columns.tolist()}

with open(os.path.join(LOG_PATH, LOG_DATA_PKL), "wb") as output_file:
    pickle.dump(data_details, output_file)

In [17]:
# Model details
model = {"model_description": "Baseline model: LinearRegression",
         "model_details": str(lr),
         "model_object": lr}

with open(os.path.join(LOG_PATH, LOG_MODEL_PKL), "wb") as output_file:
    pickle.dump(model, output_file)

In [18]:
# Performance details
regression_metrics = {"train_scores": train_scores,
                      "test_scores": test_scores}

with open(os.path.join(LOG_PATH, LOG_METRICS_PKL), "wb") as output_file:
    pickle.dump(regression_metrics, output_file)

### 2. Logging

In [19]:
# Start a new run and track
with mlflow.start_run(experiment_id=exp.experiment_id, run_name=model["model_description"]):
    # log pickle
    mlflow.log_artifacts(LOG_PATH)
    
    # Track metrics
    for metric, score in mean_test_score.items():
        mlflow.log_metric(metric, score)